In [5]:
from src.agents import Volatile
import numpy as np

In [6]:
def run_simulation(instance, hops):
    for j in range(hops):
        instance.migrate(2.989e-26)

In [3]:
volatile_1 = Volatile()
volatile_2 = Volatile()
volatile_3 = Volatile()
volatile_4 = Volatile()
volatile_5 = Volatile()
volatile_6 = Volatile()
run_simulation(volatile_1, 5000)

[1.88215954 2.64825313 3.30063014 ... 4.35060732 0.16393169 2.68505931]
[1.75610306 2.3019071  2.78627221 ... 1.61554412 2.39762496 1.50816533]
[]
